# Drilling and Real Options

In [1]:
using LinearAlgebra, BenchmarkTools, SparseArrays, Plots, Distributions
cd("/Users/junwong/Dropbox/Second Year/Kellogg - Environmental/Assignments")

## Set up state space and parameters
Ideally we'd set up a parameter space thing and unpack it in the functions, but here we are.

In [2]:
S = collect(0:80);
δ = 1/1.05
D = 3000000;
X = 100000;
profit(P, X, D) = P .* X .- D;


## State transition matrix

In [3]:
T = zeros(size(S,1), size(S,1))
shock = Normal(0, 4)

for i in 1:size(S,1)
    tomorrow_cdf = S[i] + shock
    for j in 1:size(S,1)
        if j == 1
            T[i,j] = cdf(tomorrow_cdf, S[j]+0.5)
        elseif j == size(S,1)
            T[i,j] = 1 - cdf(tomorrow_cdf, S[j]-0.5)
        else
            T[i,j] = cdf(tomorrow_cdf, S[j]+0.5) - cdf(tomorrow_cdf, S[j]-0.5)
        end
    end
end

## Iteration
The choice variable here is to wait or drill today. $g_k=1$ is drilling today, waiting is $g_k=0$. 

Given current realization of the price $P_t$, firms solve a simple problem: 

$$ \max\{P_t X - D, E_t[P_{t+1} X - D]\} $$

In [4]:
iterate_wrapper = function(S, X, D, discount, tol)
    global V = zeros(size(S,1))
    err = 1
    iter = 0 
    δ = discount
    
    while err > tol
        iter += 1
        V_old = copy(V)
        Vₙₑₓₜ = T * V 
        global V = max.(profit(S, X, D), δ .* Vₙₑₓₜ)
        global gₖ = V.==profit(S,X,D), 1, 0 # 1 is drill now, 0 is wait
        err = maximum(abs.(V .- V_old))
    end
    
    return [V, gₖ[1]] 
end;

In [5]:
# Results
results = iterate_wrapper(S, X, D, δ, 1e-5);

# Confirm that trigger price is $41
S[findfirst(results[2].==1)]

41

## Plot value function

In [6]:
# Change default font
plot_font = "Computer Modern"
default(fontfamily=plot_font)

In [7]:
plot(results[1]./1000000, legend=false, 
    ylabel="Value function (millions)", xlabel="Prices", linewidth=1.2)
plot!([41], seriestype="vline", line=(:dash, 1))
savefig("output/q3_valuefunction.pdf");